In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from chinese_calendar import is_workday, is_holiday
import pickle as pk
from Utils import *
import Metrics

TASK = "out"

#数据采用统计周期
TIME_INTERVAL = 10

#数据开始时间，1日到25日，共25天数据
DATA_START_DAY = 1
DATA_END_DAY = 25

#每天被截取的时间，早晨06:00到晚上23:30
DAYSTART = "06:00"
DAYEND = "23:30"

N_NODE = 81
#输入时间步，输出时间步
INPUT_STEP = 12
PRED_STEP = 12
HORIZON = 1

# train test 划分
TRAINRATIO = 0.8 # TRAIN + VAL
TRAINVALSPLIT = 0.125 # val_ratio = 0.8 * 0.125 = 0.1


CHANNEL = 1
BATCHSIZE = 16
LEARN = 0.001
EPOCH = 400
PATIENCE = 20

OPTIMIZER = 'RMSprop'
LOSS = 'MSE'







# read data
daystartt = datetime.datetime.strptime(DAYSTART, '%H:%M') 
dayendt = datetime.datetime.strptime(DAYEND, '%H:%M') 
day_minutes = int((dayendt - daystartt).total_seconds()/60)  #每天有多少分钟的数据
day_total_step = math.ceil(day_minutes/TIME_INTERVAL)   #对应每天有多少个step
data_in,data_out = read_file(TIME_INTERVAL,DATA_START_DAY,DATA_END_DAY,DAYSTART,DAYEND)  #data数据读取，并且截取 1号到 25号， 06:00 到 23:30的数据
if TASK is "in":
    data = data_in
else :
    data = data_out
# Normalization
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data_nor = scaler.fit_transform(data)
#data 转换成sequence
data = data.values.reshape(data.shape[0],data.shape[1],1)
seq_data = datasetToSeq_daybyday(data, INPUT_STEP,PRED_STEP, day_total_step, day=DATA_END_DAY-DATA_START_DAY+1)
seq_data = seq_data.transpose(0,3,1,2)



def getXSYS(data, mode):
    TRAIN_NUM = int(data.shape[0] * TRAINRATIO)
    train_set = data[0:TRAIN_NUM]
    test_set = data[TRAIN_NUM:]
    XS, YS = [], []
    if mode == 'TRAIN': 
        XS = train_set[:,:,0:INPUT_STEP,:]
        YS = train_set[:,:,INPUT_STEP:INPUT_STEP+PRED_STEP,:]
    elif mode == 'TEST':
        XS = test_set[:,:,0:INPUT_STEP,:]
        YS = test_set[:,:,INPUT_STEP:INPUT_STEP+PRED_STEP,:]
    return XS, YS

trainXS, trainYS = getXSYS(seq_data, 'TRAIN')
testXS, testYS = getXSYS(seq_data, 'TEST')

In [2]:
MODELNAME = 'CopyLastFrame'
PATH = './save'


# testXS = scaler.inverse_transform(testXS)
# testYS = scaler.inverse_transform(testYS)
def CopyLastSteps(XS, YS):
    return XS

def testModel(name, mode, XS, YS):
    print('TIMESTEP_IN, TIMESTEP_OUT', INPUT_STEP, PRED_STEP)
    YS_pred = CopyLastSteps(XS, YS)
    print('YS.shape, YS_pred.shape,', YS.shape, YS_pred.shape)
    np.save(PATH + '/' + MODELNAME + '_prediction.npy', YS_pred)
    np.save(PATH + '/' + MODELNAME + '_groundtruth.npy', YS)
    _, RMSE, MAE, MAPE = Metrics.evaluate(YS, YS_pred)
    print('*' * 40)
    f = open(PATH + '/' + name + '_prediction_scores.txt', 'a')
    print("all pred steps, %s, %s, RMSE, MAE, MAPE, %.10f, %.10f, %.10f" % (name, mode,RMSE, MAE, MAPE))
    f.write("all pred steps, %s, %s, RMSE, MAE, MAPE, %.10f, %.10f, %.10f\n" % (name, mode, RMSE, MAE, MAPE))
    for i in range(PRED_STEP):
        _, RMSE, MAE, MAPE = Metrics.evaluate(YS[:, :, i, :], YS_pred[:, :, i, :])
        print("%d step, %s, %s, RMSE, MAE, MAPE, %.10f, %.10f, %.10f" % (i+1, name, mode, RMSE, MAE, MAPE))
        f.write("%d step, %s, %s, RMSE, MAE, MAPE, %.10f, %.10f, %.10f\n" % (i+1, name, mode, RMSE, MAE, MAPE))
    f.close()
    
testModel(MODELNAME, 'TEST', testXS, testYS)

TIMESTEP_IN, TIMESTEP_OUT 12 12
YS.shape, YS_pred.shape, (410, 1, 12, 81) (410, 1, 12, 81)
****************************************
all pred steps, CopyLastFrame, TEST, RMSE, MAE, MAPE, 167.5728415772, 87.2330397471, 67.8563836991
1 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 203.1112080672, 104.1831376092, 66.8335903978
2 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 198.2200247943, 101.0877145438, 66.2485723593
3 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 189.6667346158, 96.9580849142, 65.6718110099
4 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 180.3138216689, 92.8230954532, 65.2185830929
5 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 171.9586572268, 89.0320084312, 64.8625545639
6 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 160.8957135955, 84.8522734116, 64.6938746642
7 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 153.2350655223, 81.6943691659, 64.8505826523
8 step, CopyLastFrame, TEST, RMSE, MAE, MAPE, 149.8259412682, 79.9797952424, 65.5404647628
9 step, CopyLastFrame, TEST, RMSE, MAE,